In [3]:
!pip install transformers datasets langchain chromadb bitsandbytes accelerate langchain_community
!pip install git+https://github.com/xlang-ai/instructor-embedding.git
!pip install sentence-transformers

  Cloning https://github.com/xlang-ai/instructor-embedding.git to /tmp/pip-req-build-yubo6uju
  Running command git clone --filter=blob:none --quiet https://github.com/xlang-ai/instructor-embedding.git /tmp/pip-req-build-yubo6uju
  Resolved https://github.com/xlang-ai/instructor-embedding.git to commit 9a9742c130cef6e028af9f3bd582204c5bcfe96e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 62.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
import os, torch, logging, time, atexit
import bitsandbytes as bnb
from getpass import getpass
from datasets import load_dataset
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, pipeline, BitsAndBytesConfig
from tqdm import tqdm

from chromadb.config import Settings

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass("Enter your Hugging Face otoken: ")
    print("HF_TOKEN is set:", "HF_TOKEN" in os.environ)

0.45.5
Using device: cuda:0


In [6]:
model_name = "meta-llama/Llama-3.2-3B"
HUGGING_FACE_TOKEN = os.getenv("HF_TOKEN")

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_dtype="nf4")

def load_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
    return model,tokenizer

model,tokenizer = load_model_and_tokenizer()

def verify_model(model,tokenizer):
    input_text = "Testin 123 123 123"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)

print(f"Model test output: {verify_model(model,tokenizer)}")



OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-3B.
403 Client Error. (Request ID: Root=1-6800bd08-6a96523e63da1aa708dbedcc;9aed1e1b-d49c-44f0-8d6d-762b47bedf72)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-3B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-3B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-3B to ask for access.

In [ ]:
def load_pubmed_qa_dataset(split="train", limit= None):
    dataset = load_dataset("pubmed_qa", "pqa_labeled", split=split)
    return dataset.select(range(min(limit,len(dataset)))) if limit else dataset


def init_chroma():
    embedding_fn = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-mpnet0base-v2",
        model_kwargs = {"device" : DEVICE}
    )
    return Chroma(
        embedding_function=embedding_fn,
        client_settings=Settings(anonymized_telemetry=False)

    )


